# PEC Monte Carlo simulation

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [ ]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

In the next few steps we import all relevant data and set up the melt initial FeO prediction model. These steps are identical to the [PEC model example](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html#)

Import melt inclusion and olivine data:

In [ ]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
olivine = mp.read_olivine(olivine_file, index_col=["name"])

Import inclusion internal pressures:

In [ ]:
pressure_file ="./data/pressure.csv"
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

Set up the melt initial FeO prediction model:

In [ ]:
wholerock_file = "./data/wholerock.csv"
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

Next, we need to set up the object that handles the random sampling of errors in the Monte Carlo simulation. This is done with the *PEC_MC_parameters* class and it includes the following parameters for error propagation:

- **melt_errors**
        
    propagate errors on melt composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **olivine_errors**

    propagate errors on olivine composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **FeOi_errors**

    propagate errors on estimate melt initial FeO contents. Fixed errors can be provided either for the whole dataset, or per inclusion. Alternatively, an [FeOi_prediction object](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html) can be provided to propagate errors on predictions models.

- **Fe3Fe2**

    Propagate errors on modelled melt Fe<sup>2+</sup>/Fe<sup>3+</sup> ratios. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

- **Kd**

    Propagate errors on modelled olivine-melt Fe-Mg partition coefficients. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

By default errors are not propagated - you explicitely need to tell MagmaPEC to when initialising the *PEC_MC_parameters* object

In this example we will use all error propagation options, which means we need to provide errors on the melt and olivine compositions. We import these from .csv files containing (fake) error data for individual inclusions and olivines.

In [ ]:
melt_errors_file = "./data/melt_errors.csv"
olivine_errors_file = "./data/olivine_errors.csv"

melt_errors = pd.read_csv(melt_errors_file, index_col=[0])
olivine_errors = pd.read_csv(olivine_errors_file, index_col=[0])

Make absolutely sure that the elements in the error data have identical sorting to the melt and olivine dataframes, otherwise errors will be applied to the wrong elements. 
We can force this by sorting the columns of the error dataframes (or series) with the *elements* attributes of the melt and olivine MagmaFrames:

In [ ]:
melt_errors = melt_errors[melt.elements]
olivine_errors = olivine_errors[olivine.elements]

Here's what they look like:

In [ ]:
melt_errors.head()

In [ ]:
olivine_errors.head()

Together with the *FeOi_prediction* object, we pass these as arguments to the *PEC_MC_parameters* object. We also set *Fe3Fe2* and *Kd* to *True* in order to propagate their model errors as well.

In [ ]:
mc_parameters = mpc.PEC_MC_parameters(melt_errors=melt_errors, olivine_errors=olivine_errors, FeOi_errors=FeOi_predict, Fe3Fe2=True, Kd=True)

Now we can create the Monte Carlo model with the *PEC_MC* object

In [ ]:
pec_mc_model = mpc.PEC_MC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeOi_predict, MC_parameters=mc_parameters)

And finally run the Monte Carlo simulation with *n* iterations

In [ ]:
pec_mc_model.run(n=30)

Equilibrating ... |███████████████████████████   | 90% [9/10] in 4:23 

Results are stored internally in the following attributes:

- **pec**: pandas DataFrame

    Average PEC extents (%) of the MC model and their one standard deviation errors.

- **inclusions_corr**: MagmaPandas Melt frame

    Averages of corrected melt inclusion compositions

- **inclusions_stddev**: pandas DataFrame

    One standard deviation errors on inclusions_corr

- **pec_MC**: pandas DataFrame

    PEC extents for individual iterations

- **inclusions_MC**: dictionary of MagmaPandas Melt frames

    corrected melt inclusion compositions for individual iterations.

In [ ]:
pec = pec_mc_model.pec
inclusions_corrected = pec_mc_model.inclusions_corr
inclusions_errors = pec_mc_model.inclusions_stddev

pec_mc = pec_mc_model.pec_MC
inclusions_MC = pec_mc_model.inclusions_MC

In [ ]:
pec

In [ ]:
inclusions_corrected

In [ ]:
inclusions_errors

In [ ]:
pec_mc

In [ ]:
inclusions_MC["PI032-04-01"]